<p><img src="imagenes/letragrama-rgb-150.jpg" width="380" align="left"><img src="imagenes/logoLCCazul.jpg" width="170" align="right"></p>

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Red neuronal convolucional simple con tensorflow

[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 28 de noviembre de 2017.

En esta libreta intento explicar de forma simple como funciona el método sencillo de red neuronal convolucional usando *Tensorflow*. La explicación de la red convolucional ya está dada y en esta libreta solamente se explican los pasos necesarios para implementar una red convolucional simple explicando cada uno de sus pasos. Es básicamente una traducción, y puesta en forma de libreta del tutorial que viene en la página de *Tensorflow* y que se puede consultar [aquí](https://www.tensorflow.org/tutorials/layers).


Las redes convolucionales (CNN) son actualmente la arquitectura que mejores resultados he dado en problemas de reconocimiento de objetos en imágenes. La idea generar es aplicar una serie de filtros a la imagen original para extraer (por medio de aprendizaje) características de alto nivel, las cuales puedan ser útiles para el proceso de clasificacion de objetos en imágenes. Las CNN están conformadas típicamente por:

1. **Unidades convolucionales**. Las unidades convolucionales aplican filtros a la imágen y la convierten en un conjunto de imágenes tratadas a dimensión reducida, poniendo el acento en diferentes características. Estas unidades se componen a su vez de dos capas:

    1. **Capa convolucional**. La capa convolucional es uno o varios filtros los cuales se aplican a cada subregión al rededor de cada pixel con el fin de obtener una imagen tratada. Típicamente un filtro convolucional es un filtro lineal al cual al final se le aplica una no linearidad (típicamente una ReLU). Un factor importante del filtro es que los valores de la ventana del filtro se mantienen fijos a lo largo de toda la imagen (esto es, comparten *pesos* entre las diferentes subregiones).
    
    2. **Una capa de *pooling* **. La capa de pooling ayuda a reducir el muestreo de las imágenes generadas por los filtros, y reducir la dimensionalidad. Se basa en la idea que las salidas de los filtros se deben de parecer localmente y solo interesa un valor, el más significativo. El método de *pooling* más comúnes el máximo.
    
2. **Capas densas**. Estas se aplican después de una o varias capas convolucionales y son capas tipo *feedforward* donde todas las neuronas se conectn con todas. Es básicamente la etapa de clasificación, mientras que la anterior era la etápa de preprocesamiento de la imágen.

Por último, y en problemas de clasificación, la salida es una unidad *softmax* que permita discriminar entre diferentes tipos de salida posibles.






In [2]:
import numpy as np
import tensorflow as tf


La CNN que vamos a desarrollar (que es la arquitectura usada por LeNet) para clasificar el famosisimo y ya cansino conjunto de datos MNIST tiene la siguiente arquitectura:

1. Una unidad convolucional que contiene:
    1. 32 filtros de 5$\times$5 (en pixeles al rededor del pixel a filtrar), con activación ReLU.
    2. Una capa de *pooling* usando el máximo de una ventana de 2$\times$2 sin translape, por cada imágen generada.
2. Otra unidad convolucional que contiene:
    1. 64 filtros de 5$\times$5 con activación ReLU.
    2. Una capa de *pooling* usando el máximo de una ventana de 2$\times$2 sin translape, por cada imágen generada.
3. Una capa de neuronas tipo *feedforward* con 1,024 neuronas con activación ReLU.
4. Una capa de salida de 10 neuronas que compone una unidad *Softmax*.

Con el fin de mejorar la generalización, a la capa 3 (la capa oculta en *feedforward* se le aplica (en el aprendizaje) el método de *dropout* con una tasa de 0.4 (esto es, cualquier neurona puede ser retirada en forma aleatoria durante el aprendizaje con una probabilidad de 0.4).

Vamos a armar este modelo con el uso de las facilidades que da `tf.layers`, que nos permiten hacer una capa de una CNN en una sola linea. Rápido, bonito y efectivo. Vamos a ver como se diseña la arquitectura de la CNN descrita:

In [3]:
def modelo_cnn_mnist(features, labels, mode):
    """
    Modelo de la CNN descrita arriba
    """
    
    # Capa de entrada: 
    # Convierte un vector de datos en un tensor [batch, ancho, alto, canales]
    # Donde canales son las capas de la imagen (por ejemplo RGB)
    # En este caso las imagenes son en BW (un solo canal)
    # y de 24 x 24 pixeles.
    # 
    # batch = -1 significa que lo calcula al vuelo para completar.
    capa_entrada = tf.reshape(features["x"], [-1, 28, 28, 1])

    # 1ra capa convolucional
    # 32 filtros de 5x5 y salida ReLU
    # padding puede ser "same", lo que implica que calcula el filtro
    # inclusive para las orillas, si padding=valid entonces no calcula 
    # el filtro en las orillas, completando con 0s.
    # 
    # En este caso si la entrada a la capa es de dimensión [-1, 28, 28, 1]
    # la salida de la capa es [-1, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=capa_entrada,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # 1ra capa max pool, sin translape (strides = poolsize[0] = poolsize[1])
    # Si la entrada a la capa era de dimensiones [-1, 28, 28, 32]
    # la salida es [-1, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=[2, 2], 
        strides=2)

    # 2da capa convolucional
    # 64 filtros de 5x5 y salida ReLU
    # Si la entrada a la capa es de la forma [-1, 14, 14, 32]
    # La salida es de la forma [-1, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # 2da capa max pool, sin translape (strides = poolsize[0] = poolsize[1])
    # Si la entrada a la capa era de dimensiones [-1, 14, 14, 64]
    # la salida es [-1, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2, 
        pool_size=[2, 2], 
        strides=2)


    # Capa densamente conectada con 1024 unidades linealmente rectificadas
    pool2_estirado = tf.reshape(pool2, [-1, 7 * 7 * 64])
    capa_densa = tf.layers.dense(
        inputs=pool2_estirado, 
        units=1024, 
        activation=tf.nn.relu)

    # Agregamos el dropout aqui (esto ya empieza a ser bizarro)
    # En los modelos se pone la arquitectura, la forma de aprendizaje y estimación,
    # así que si requerimos dropout en una capa hay que ponerlo aqui en forma explicita.
    #
    # Notese que decimos que solo si estamos en modo entrenamiento es que aplicamos el dropout
    dropout = tf.layers.dropout(
        inputs=capa_densa, 
        rate=0.4, 
        training=mode == tf.estimator.ModeKeys.TRAIN)

    # Capa de salida
    # Dimensión de la entrada: [-1, 1024]
    # y la salida es de [-1, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    
    # Listo! Ya esta la arquitectura general, ahora hay que definir claramente las cosas 
    # para los diferentes modos de operación (así funcionan los modelos que se usan preentrenados)
    # Los modos de operación son 3: PREDICT, TRAIN, EVAL.

    predicciones = {
        # Realiza la predicción (para los modos preestablecidos PREDICT y EVAL)
        # En este caso nos interesa la clase retenida y la probabilidad de cada una
        "clases": tf.argmax(input=logits, axis=1),
        "probabilidades": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # Y para la predicción tenemos lo necesario
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predicciones)

    # Calcula la función de pérdida (para los modos TRAIN y EVAL)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)

    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, 
        logits=logits)

    # Completa lo necesario para el modo TRAIN (método de optimización)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizador = tf.train.GradientDescentOptimizer(learning_rate=0.001)

        entrenamiento = optimizador.minimize(loss=loss, 
                                             global_step=tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode=mode, 
                                          loss=loss, 
                                          train_op=entrenamiento)

    # Acaba con las operaciones del modo EVAL
    evaluacion = {
        "accuracy": tf.metrics.accuracy(labels=labels, 
                                        predictions=predicciones["clases"])}
    return tf.estimator.EstimatorSpec(mode=mode, 
                                      loss=loss, 
                                      eval_metric_ops=evaluacion)


Como podemos ver, acabamos de definir un modelo completo como función, el cual debe de llevar **con esos mismos nombres**, las entradas *features*, *labels*, *mode* (si las traducen al español, se enclocha).

El modelo debe de incluir lo siguiente:

1. La arquitectura bien definida desde las entradas a las salidas, los tipos de capas, número de neuronas y form de procesar.

2. El método de evaluación utilizando salida *softmax* y asumiendo que la clase mas probable es la clase seleccionada.

3. El método de aprendizaje (descenso de gradiente con tasa de aprendizaje de 0.001). 

4. El método de medición de error (o perdida) utilizando la entropía.

5. La función de evaluación, basada en la exactitud (una de las pruebas más básicas posibles).

Las operaciones necesarias para el modo PREDICT son 1 y 2, mientras que las operaciones necesarias para el modo TRAIN son 1, 3 y 4, y para el modo TEST se requieren 1, 2 y 5. Con el fin de ahorrar en tiempo de computo, solamente se asignan las operaciones que importan para cada modo.

Vamos a aplicar para aprendizaje y reconocimiento, en el conjunto de base de pruebas que empieza a aburrirnos.

In [4]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

datos_entrenamiento = mnist.train.images  
salidas_entrenamiento = np.asarray(mnist.train.labels, dtype=np.int32)

datos_prueba = mnist.test.images  # Returns np.array
salidas_prueba = np.asarray(mnist.test.labels, dtype=np.int32)


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


Así es, tenemos la horrorosa base de datos de MNIST, a la cual le vamos a aplicar nuestro modelo

In [5]:
# El modelo que vamos a utilizar y la dirección para guardar el log
clasificador = tf.estimator.Estimator(
    model_fn=modelo_cnn_mnist, 
    model_dir="./cnn_log")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './cnn_log', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11e4de240>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Ahora vamos a entrenar, y para eso vamos a guardar en un registro (log) el proceso de aprendizaje, por lo que vamos a hacer un log descriptivo (*verbose*), así como definir cuales son las entradas con las que se va a aprender.

In [ ]:
# Para guardar y desplegar la información a lo largo del aprendizaje
tf.logging.set_verbosity(tf.logging.INFO)
tensors_to_log = {"probabilidades": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, 
                                          every_n_iter=50)


# La función que define que se le pone al entrenamiento

entrada_entrenamiento = tf.estimator.inputs.numpy_input_fn(
    x={"x": datos_entrenamiento},
    y=salidas_entrenamiento,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

clasificador.train(
    input_fn=entrada_entrenamiento,
    steps=20000,
    hooks=[logging_hook])



INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./cnn_log/model.ckpt-3991
INFO:tensorflow:Saving checkpoints for 3992 into ./cnn_log/model.ckpt.
INFO:tensorflow:probabilidades = [[ 0.00008043  0.00000978  0.00004027  0.00006431  0.00000177  0.00000501
   0.00000001  0.99843103  0.00013484  0.00123257]
 [ 0.00000065  0.00006022  0.99935371  0.00043985  0.00001259  0.00000187
   0.00010731  0.00000009  0.0000221   0.00000164]
 [ 0.00469688  0.04700846  0.07066615  0.20129259  0.00302556  0.00815385
   0.00097884  0.51171464  0.05939652  0.09306655]
 [ 0.0000149   0.00191921  0.00004134  0.00044791  0.00001054  0.00001727
   0.00000033  0.98488206  0.0001047   0.01256162]
 [ 0.03748242  0.00066098  0.00028717  0.00049523  0.0034732   0.7946952
   0.00127812  0.05623363  0.07576507  0.02962901]
 [ 0.00037427  0.0702659   0.24463998  0.648197    0.00001958  0.0109463
   0.00139612  0.00205695  0.0214647   0.00063922]
 [ 0.00002656  0.00002766  0.0002709

INFO:tensorflow:loss = 0.316868, step = 3992
INFO:tensorflow:probabilidades = [[ 0.00025117  0.00000241  0.0000063   0.00000534  0.58708531  0.00519439
   0.00395366  0.0151846   0.10633048  0.28198645]
 [ 0.00553258  0.71549112  0.01835331  0.01041364  0.00250865  0.09630632
   0.10470578  0.00069385  0.04427293  0.00172183]
 [ 0.00608331  0.02276707  0.62113822  0.11492655  0.00008233  0.00093787
   0.00088269  0.00402876  0.22609088  0.00306237]
 [ 0.00773106  0.00000094  0.00008103  0.82986432  0.0080126   0.02053312
   0.00000732  0.01831657  0.06390418  0.05154892]
 [ 0.999331    0.          0.00009066  0.00000541  0.00000001  0.00026153
   0.00000247  0.00000286  0.00029233  0.00001374]
 [ 0.00010045  0.00004254  0.00000418  0.00101524  0.02271405  0.00082532
   0.0001036   0.02757457  0.00062837  0.94699174]
 [ 0.92142922  0.00000391  0.00108659  0.03343367  0.00049056  0.02682843
   0.000027    0.00191908  0.00063912  0.01414248]
 [ 0.00001976  0.00067821  0.00003621  0.001839

INFO:tensorflow:global_step/sec: 2.6713
INFO:tensorflow:probabilidades = [[ 0.00011615  0.97691071  0.00203041  0.00489327  0.00021861  0.00148457
   0.00087256  0.00248653  0.00975876  0.00122844]
 [ 0.97718757  0.00000002  0.00433108  0.00965908  0.00000005  0.00844166
   0.00000371  0.00024533  0.00010496  0.00002658]
 [ 0.00698075  0.00040711  0.0001252   0.00240126  0.00907953  0.03074981
   0.00009018  0.62102371  0.00464297  0.32449943]
 [ 0.69227314  0.00042028  0.00244229  0.02363345  0.002257    0.13860737
   0.13509041  0.00089087  0.00387293  0.00051233]
 [ 0.00005918  0.0000737   0.00003519  0.00015695  0.05312481  0.00140782
   0.00005111  0.01426356  0.03173232  0.89909542]
 [ 0.00161477  0.00000314  0.00003577  0.00040524  0.00013621  0.00015516
   0.00000704  0.99450004  0.00012786  0.00301478]
 [ 0.00072583  0.0000076   0.00004689  0.00018854  0.00008592  0.0015619
   0.00000397  0.8579753   0.00226349  0.13714068]
 [ 0.00002274  0.96154344  0.00686177  0.00357926  0.

INFO:tensorflow:loss = 0.341432, step = 4092 (37.429 sec)
INFO:tensorflow:probabilidades = [[ 0.00011008  0.00256629  0.00024242  0.00258669  0.48441994  0.00212395
   0.00018988  0.0229817   0.0228984   0.46188068]
 [ 0.31146017  0.00008805  0.07477435  0.05161285  0.0017662   0.33384889
   0.1294478   0.0075938   0.07587278  0.01353506]
 [ 0.00010518  0.00000911  0.00160499  0.00445469  0.02550914  0.00050102
   0.00046146  0.01671901  0.00632792  0.94430745]
 [ 0.00009634  0.97260654  0.00396149  0.0052622   0.00170351  0.00112733
   0.00192907  0.00472699  0.00558755  0.002999  ]
 [ 0.00079327  0.00159178  0.00008923  0.01127899  0.01497946  0.00430377
   0.00001754  0.60040426  0.01006916  0.35647249]
 [ 0.00000079  0.00000247  0.00000018  0.00014919  0.00000874  0.00010069
   0.00000005  0.99867451  0.00002396  0.00103958]
 [ 0.0032399   0.01079026  0.00064616  0.00155408  0.00064904  0.01085279
   0.00054009  0.00250722  0.9583165   0.01090404]
 [ 0.00281039  0.01223087  0.00221

INFO:tensorflow:global_step/sec: 2.52634
INFO:tensorflow:probabilidades = [[ 0.00230474  0.00000001  0.00000272  0.01282644  0.00000797  0.97738385
   0.00000001  0.00297413  0.00161816  0.00288192]
 [ 0.00006433  0.00080241  0.00122144  0.97435057  0.00004091  0.00572839
   0.00001354  0.0007369   0.01588405  0.00115748]
 [ 0.00632177  0.00000011  0.00070112  0.00000469  0.00060907  0.00027549
   0.99162334  0.00000888  0.00001221  0.00044342]
 [ 0.00224983  0.00332715  0.00046421  0.00611875  0.0027232   0.85540092
   0.00096695  0.00132895  0.12213928  0.00528076]
 [ 0.01498084  0.00004946  0.00274338  0.00608333  0.00021915  0.5430823
   0.30379584  0.00008834  0.12864847  0.00030893]
 [ 0.00079841  0.79730886  0.01476095  0.04978292  0.00152228  0.08626281
   0.02122615  0.00256243  0.02237877  0.00339644]
 [ 0.00016539  0.00005232  0.01488176  0.00000841  0.17742783  0.00005215
   0.80570447  0.00000316  0.00143566  0.00026886]
 [ 0.00000869  0.00001863  0.00008376  0.00008219  0

INFO:tensorflow:loss = 0.458758, step = 4192 (39.587 sec)
INFO:tensorflow:probabilidades = [[ 0.00036004  0.01051435  0.00228655  0.01120518  0.57437605  0.01173517
   0.00778291  0.00514488  0.02808327  0.34851161]
 [ 0.00412574  0.00001155  0.00056792  0.02230761  0.00052882  0.96021092
   0.00577635  0.00002342  0.00611037  0.00033731]
 [ 0.00289856  0.0152758   0.00727733  0.59701127  0.00016724  0.16780725
   0.00015652  0.02426868  0.18289439  0.00224296]
 [ 0.00820905  0.00022861  0.01507052  0.00133993  0.21905109  0.01160518
   0.73668194  0.00026671  0.00246588  0.00508125]
 [ 0.0000757   0.00036219  0.05253964  0.00042637  0.79079533  0.0002581
   0.1386988   0.00105312  0.0011319   0.01465891]
 [ 0.00002019  0.00128262  0.99631965  0.00086784  0.00000158  0.00001831
   0.00027271  0.00000034  0.00121297  0.00000389]
 [ 0.00009234  0.979599    0.00097417  0.00772074  0.00085155  0.00249762
   0.00278142  0.00135802  0.00339345  0.00073171]
 [ 0.96356487  0.00000008  0.001194

INFO:tensorflow:global_step/sec: 2.70777
INFO:tensorflow:probabilidades = [[ 0.00066554  0.00015337  0.97006756  0.00020357  0.01691434  0.00009043
   0.0090282   0.00007713  0.00079101  0.00200874]
 [ 0.00000495  0.00001958  0.99221694  0.00003868  0.00003977  0.0000002
   0.00765443  0.00000001  0.00002446  0.00000098]
 [ 0.00005652  0.93772066  0.00278986  0.00214388  0.00655433  0.00018977
   0.00181723  0.00580769  0.02207232  0.02084765]
 [ 0.08678015  0.00392805  0.0661488   0.75154036  0.00466109  0.04107066
   0.01941002  0.00013626  0.02244526  0.00387937]
 [ 0.00042484  0.00059769  0.04440322  0.88165992  0.00045122  0.00875347
   0.0004518   0.0019821   0.0547855   0.00649021]
 [ 0.00001464  0.99542516  0.00040092  0.00057121  0.00008686  0.00059695
   0.00117371  0.0003286   0.00119879  0.0002031 ]
 [ 0.99536127  0.00000002  0.00176347  0.0009013   0.00000014  0.0017772
   0.00016474  0.00001472  0.00001382  0.00000344]
 [ 0.00179056  0.00000072  0.00050927  0.00133349  0.

INFO:tensorflow:loss = 0.324717, step = 4292 (36.927 sec)
INFO:tensorflow:probabilidades = [[ 0.00000233  0.00000405  0.00001177  0.00052577  0.00005941  0.00003859
   0.00000003  0.98541981  0.00030786  0.01363033]
 [ 0.01951842  0.00055758  0.7368573   0.00021526  0.07825818  0.00113987
   0.1578308   0.00000145  0.00542507  0.0001961 ]
 [ 0.00314983  0.00000184  0.9934454   0.00074385  0.00004298  0.00000273
   0.00255804  0.00000107  0.00005332  0.00000095]
 [ 0.00022647  0.00000017  0.00092174  0.00000216  0.00001238  0.00003613
   0.99874866  0.00000001  0.00004969  0.00000254]
 [ 0.0000172   0.0000186   0.00002076  0.00041684  0.00022969  0.00005289
   0.00000088  0.9901315   0.00017167  0.00893983]
 [ 0.00759734  0.00643735  0.04173028  0.00198817  0.00079731  0.04429375
   0.89186734  0.00014147  0.00498046  0.00016667]
 [ 0.00000422  0.00000007  0.00001608  0.99928707  0.          0.00067525
   0.00000004  0.00000338  0.00001139  0.00000255]
 [ 0.02946112  0.0803223   0.07415

INFO:tensorflow:global_step/sec: 2.73838
INFO:tensorflow:probabilidades = [[ 0.02051131  0.00000078  0.00115393  0.00007914  0.79706299  0.00928879
   0.06996837  0.00030887  0.00208837  0.09953748]
 [ 0.17971244  0.00483269  0.01003133  0.00296718  0.00410757  0.7373805
   0.02289051  0.00429236  0.03008602  0.00369941]
 [ 0.00051028  0.00000232  0.00000486  0.00209198  0.00002494  0.99217492
   0.00006316  0.00000405  0.00505254  0.000071  ]
 [ 0.00001448  0.00000028  0.99876344  0.00106379  0.00001499  0.00000111
   0.00002417  0.00000058  0.00011448  0.00000263]
 [ 0.00152393  0.9272725   0.01848874  0.01726276  0.00117439  0.00370375
   0.00314116  0.00353097  0.02238994  0.00151187]
 [ 0.04789644  0.00000002  0.19041337  0.00038535  0.00085879  0.04055101
   0.14694315  0.0000003   0.57234871  0.00060287]
 [ 0.00046104  0.00000925  0.00290054  0.00003186  0.01799582  0.00073894
   0.97715944  0.00000053  0.00066939  0.00003331]
 [ 0.01054065  0.00000503  0.00048551  0.07273907  0

INFO:tensorflow:loss = 0.27649, step = 4392 (36.519 sec)
INFO:tensorflow:probabilidades = [[ 0.00007079  0.92252636  0.00208082  0.03994841  0.00003161  0.00780407
   0.00070882  0.00441385  0.01922991  0.00318529]
 [ 0.000093    0.00000275  0.00043819  0.00001203  0.00151893  0.00006151
   0.99764508  0.00000421  0.00018681  0.00003745]
 [ 0.00014883  0.00011405  0.00373364  0.9663617   0.00001275  0.01034614
   0.00014921  0.00007079  0.0187842   0.00027868]
 [ 0.00001546  0.00002324  0.99827242  0.00024459  0.00008701  0.00008457
   0.00080626  0.00000007  0.00046024  0.00000618]
 [ 0.00047138  0.00011643  0.00119225  0.00112246  0.0067951   0.00097276
   0.00024557  0.0557549   0.0094147   0.92391455]
 [ 0.00120439  0.89131039  0.03431747  0.01517889  0.00202688  0.00212432
   0.00247428  0.01434333  0.03136265  0.00565746]
 [ 0.00029685  0.04287241  0.00075969  0.02003472  0.04418552  0.0066348
   0.00019422  0.40162355  0.05367385  0.4297244 ]
 [ 0.00361634  0.00000314  0.0002360

Y ahora aplicamos los modos de predicción y prueba

In [8]:
entrada_prueba = tf.estimator.inputs.numpy_input_fn(
    x={"x": datos_prueba},
    y=salidas_prueba,
    num_epochs=1,
    shuffle=False)

eval_results = clasificador.evaluate(input_fn=entrada_prueba)
  
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-04-04-19:58:28
INFO:tensorflow:Restoring parameters from ./cnn_log/model.ckpt-3991
INFO:tensorflow:Finished evaluation at 2018-04-04-19:58:40
INFO:tensorflow:Saving dict for global step 3991: accuracy = 0.9175, global_step = 3991, loss = 0.305909
{'accuracy': 0.91750002, 'loss': 0.30590925, 'global_step': 3991}
